# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."  
We will show how to use them here, in the context of pinkfish.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import talib
from talib.abstract import *
from talib import MA_Type

import pinkfish as pf

# Format price data
pd.options.display.float_format = '{:0.3f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [4]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from cache, if available.

In [5]:
ts = pf.fetch_timeseries(symbol)
ts.tail()

dir_name data


open    high     low   close  adj_close    volume
date                                                           
2021-11-09 469.320 469.570 465.880 467.380    467.380  51149100
2021-11-10 465.580 467.380 462.040 463.620    463.620  69429700
2021-11-11 465.210 465.290 463.750 463.770    463.770  34848500
2021-11-12 465.120 467.860 464.110 467.270    467.270  53423300
2021-11-15 468.640 468.788 466.230 467.000    467.000  25962042

Select timeseries between start and end.

In [6]:
ts = pf.select_tradeperiod(ts, start, end)
ts.head()

open    high     low   close  adj_close    volume
date                                                           
2017-01-03 225.040 225.830 223.880 225.240    206.872  91366500
2017-01-04 225.620 226.750 225.610 226.580    208.103  78744400
2017-01-05 226.270 226.580 225.480 226.400    207.937  78379000
2017-01-06 226.530 227.750 225.900 227.210    208.681  71559900
2017-01-09 226.910 227.070 226.420 226.460    207.992  46939700

### Get info about TA-Lib

In [7]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [8]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [9]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [10]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [11]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [12]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [13]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [14]:
sma = SMA(ts)
sma.tail()

date
2021-11-09   448.708
2021-11-10   449.680
2021-11-11   450.834
2021-11-12   451.935
2021-11-15   453.214
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [15]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2021-11-09   422.929
2021-11-10   423.375
2021-11-11   423.806
2021-11-12   424.292
2021-11-15   424.746
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [16]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2021-11-09   422.748
2021-11-10   423.175
2021-11-11   423.619
2021-11-12   424.067
2021-11-15   424.541
dtype: float64

### Add a technical indicator to a pinkfish timeseries

In [17]:
ts['sma200'] = sma200
ts.tail()

open    high     low   close  adj_close    volume  sma200
date                                                                   
2021-11-09 469.320 469.570 465.880 467.380    467.380  51149100 422.748
2021-11-10 465.580 467.380 462.040 463.620    463.620  69429700 423.175
2021-11-11 465.210 465.290 463.750 463.770    463.770  34848500 423.619
2021-11-12 465.120 467.860 464.110 467.270    467.270  53423300 424.067
2021-11-15 468.640 468.788 466.230 467.000    467.000  25962042 424.541

### Try another one

Commodity Channel Index

In [18]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [19]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [20]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [21]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

open    high     low   close  adj_close    volume  sma200    cci
date                                                                          
2021-11-09 469.320 469.570 465.880 467.380    467.380  51149100 422.748 97.694
2021-11-10 465.580 467.380 462.040 463.620    463.620  69429700 423.175 45.428
2021-11-11 465.210 465.290 463.750 463.770    463.770  34848500 423.619 36.055
2021-11-12 465.120 467.860 464.110 467.270    467.270  53423300 424.067 60.353
2021-11-15 468.640 468.788 466.230 467.000    467.000  25962042 424.541 68.563

### Now for something a little more difficult

Bollinger Bands

In [22]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [23]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [24]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [25]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [26]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [27]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [28]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

open    high     low   close  adj_close    volume  sma200  \
date                                                                      
2021-11-09 469.320 469.570 465.880 467.380    467.380  51149100 422.748   
2021-11-10 465.580 467.380 462.040 463.620    463.620  69429700 423.175   
2021-11-11 465.210 465.290 463.750 463.770    463.770  34848500 423.619   
2021-11-12 465.120 467.860 464.110 467.270    467.270  53423300 424.067   
2021-11-15 468.640 468.788 466.230 467.000    467.000  25962042 424.541   

              cci   upper  middle   lower  
date                                       
2021-11-09 97.694 475.231 457.501 439.771  
2021-11-10 45.428 473.252 458.084 442.915  
2021-11-11 36.055 472.346 458.625 444.904  
2021-11-12 60.353 472.351 459.448 446.546  
2021-11-15 68.563 472.096 460.168 448.240

In [29]:
print(MOM)

MOM([input_arrays], [timeperiod=10])

Momentum (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 10
Outputs:
    real


In [30]:
mom10 = MOM(ts, timeperiod=10)
mom10.tail()

date
2021-11-09   11.420
2021-11-10    9.680
2021-11-11    5.450
2021-11-12    8.020
2021-11-15    6.960
dtype: float64

In [31]:
ts['mom10'] = mom10
ts.head(50)

open    high     low   close  adj_close     volume  sma200  \
date                                                                       
2017-01-03 225.040 225.830 223.880 225.240    206.872   91366500     NaN   
2017-01-04 225.620 226.750 225.610 226.580    208.103   78744400     NaN   
2017-01-05 226.270 226.580 225.480 226.400    207.937   78379000     NaN   
2017-01-06 226.530 227.750 225.900 227.210    208.681   71559900     NaN   
2017-01-09 226.910 227.070 226.420 226.460    207.992   46939700     NaN   
2017-01-10 226.480 227.450 226.010 226.460    207.992   63771900     NaN   
2017-01-11 226.360 227.100 225.590 227.100    208.580   74650000     NaN   
2017-01-12 226.500 226.750 224.960 226.530    208.057   72113200     NaN   
2017-01-13 226.730 227.400 226.690 227.050    208.534   62717900     NaN   
2017-01-17 226.310 226.780 225.800 226.250    207.799   61240800     NaN   
2017-01-18 226.540 226.800 225.900 226.750    208.259   54793300     NaN   
2017-01-19 226.840 227.000 225.410 225.910    207.487   66608800     NaN   
2017-01-20 226.700 227.310 225.970 226.740    208.249  129168600     NaN   
2017-01-23 226.740 226.810 225.270 226.150    207.708   75061600     NaN   
2017-01-24 226.400 228.080 226.270 227.600    209.039   95555300     NaN   
2017-01-25 228.700 229.570 228.510 229.570    210.849   84437700     NaN   
2017-01-26 229.400 229.710 229.010 229.330    210.628   59970700     NaN   
2017-01-27 229.420 229.590 228.760 228.970    210.298   59711100     NaN   
2017-01-30 228.170 228.200 226.410 227.550    208.993   79737300     NaN   
2017-01-31 226.980 227.600 226.320 227.530    208.975   75880800     NaN   
2017-02-01 227.530 228.590 226.940 227.620    209.058   79117700     NaN   
2017-02-02 227.620 228.100 226.820 227.770    209.195   69657600     NaN   
2017-02-03 228.820 229.550 228.460 229.340    210.637   80563200     NaN   
2017-02-06 228.870 229.330 228.540 228.930    210.261   57790100     NaN   
2017-02-07 229.380 229.660 228.720 228.940    210.270   57931200     NaN   
2017-02-08 228.940 229.390 228.310 229.240    210.546   51566200     NaN   
2017-02-09 229.240 230.950 229.240 230.600    211.795   65955200     NaN   
2017-02-10 231.000 231.770 230.620 231.510    212.630   66015900     NaN   
2017-02-13 232.080 233.070 232.050 232.770    213.788   55182100     NaN   
2017-02-14 232.560 233.710 232.160 233.700    214.642   71109000     NaN   
2017-02-15 233.450 235.140 233.390 234.920    215.762   86785800     NaN   
2017-02-16 234.950 235.160 233.850 234.720    215.579   84722400     NaN   
2017-02-17 233.950 235.090 233.930 235.090    215.918   77204100     NaN   
2017-02-21 235.520 236.690 235.510 236.490    217.204   88946100     NaN   
2017-02-22 236.020 236.540 235.830 236.280    217.011   62115200     NaN   
2017-02-23 236.880 236.900 235.560 236.440    217.158   74615900     NaN   
2017-02-24 235.460 236.790 235.410 236.740    217.434   82381600     NaN   
2017-02-27 236.640 237.310 236.350 237.110    217.774   56515400     NaN   
2017-02-28 236.670 236.950 236.020 236.470    217.186   96961900     NaN   
2017-03-01 238.390 240.320 238.370 239.780    220.226  149158200     NaN   
2017-03-02 239.560 239.570 238.210 238.270    218.839   70246000     NaN   
2017-03-03 238.170 238.610 237.730 238.420    218.977   81974300     NaN   
2017-03-06 237.500 238.120 237.010 237.710    218.325   55391500     NaN   
2017-03-07 237.360 237.770 236.760 237.000    217.673   65103700     NaN   
2017-03-08 237.340 237.640 236.400 236.560    217.269   78168800     NaN   
2017-03-09 236.700 237.240 235.740 236.860    217.544   90683900     NaN   
2017-03-10 237.970 238.020 236.590 237.690    218.306   81991700     NaN   
2017-03-13 237.620 237.860 237.240 237.810    218.417   57256800     NaN   
2017-03-14 237.180 237.240 236.190 236.900    217.581   59880800     NaN   
2017-03-15 237.560 239.440 237.290 238.950    219.464   96081800     NaN   

               cci   upper  middle   lower  mom10  
date     

In [32]:
m1 = ts['close'].pct_change(periods=10)
ts['m1'] = m1
ts.head(50)

open    high     low   close  adj_close     volume  sma200  \
date                                                                       
2017-01-03 225.040 225.830 223.880 225.240    206.872   91366500     NaN   
2017-01-04 225.620 226.750 225.610 226.580    208.103   78744400     NaN   
2017-01-05 226.270 226.580 225.480 226.400    207.937   78379000     NaN   
2017-01-06 226.530 227.750 225.900 227.210    208.681   71559900     NaN   
2017-01-09 226.910 227.070 226.420 226.460    207.992   46939700     NaN   
2017-01-10 226.480 227.450 226.010 226.460    207.992   63771900     NaN   
2017-01-11 226.360 227.100 225.590 227.100    208.580   74650000     NaN   
2017-01-12 226.500 226.750 224.960 226.530    208.057   72113200     NaN   
2017-01-13 226.730 227.400 226.690 227.050    208.534   62717900     NaN   
2017-01-17 226.310 226.780 225.800 226.250    207.799   61240800     NaN   
2017-01-18 226.540 226.800 225.900 226.750    208.259   54793300     NaN   
2017-01-19 226.840 227.000 225.410 225.910    207.487   66608800     NaN   
2017-01-20 226.700 227.310 225.970 226.740    208.249  129168600     NaN   
2017-01-23 226.740 226.810 225.270 226.150    207.708   75061600     NaN   
2017-01-24 226.400 228.080 226.270 227.600    209.039   95555300     NaN   
2017-01-25 228.700 229.570 228.510 229.570    210.849   84437700     NaN   
2017-01-26 229.400 229.710 229.010 229.330    210.628   59970700     NaN   
2017-01-27 229.420 229.590 228.760 228.970    210.298   59711100     NaN   
2017-01-30 228.170 228.200 226.410 227.550    208.993   79737300     NaN   
2017-01-31 226.980 227.600 226.320 227.530    208.975   75880800     NaN   
2017-02-01 227.530 228.590 226.940 227.620    209.058   79117700     NaN   
2017-02-02 227.620 228.100 226.820 227.770    209.195   69657600     NaN   
2017-02-03 228.820 229.550 228.460 229.340    210.637   80563200     NaN   
2017-02-06 228.870 229.330 228.540 228.930    210.261   57790100     NaN   
2017-02-07 229.380 229.660 228.720 228.940    210.270   57931200     NaN   
2017-02-08 228.940 229.390 228.310 229.240    210.546   51566200     NaN   
2017-02-09 229.240 230.950 229.240 230.600    211.795   65955200     NaN   
2017-02-10 231.000 231.770 230.620 231.510    212.630   66015900     NaN   
2017-02-13 232.080 233.070 232.050 232.770    213.788   55182100     NaN   
2017-02-14 232.560 233.710 232.160 233.700    214.642   71109000     NaN   
2017-02-15 233.450 235.140 233.390 234.920    215.762   86785800     NaN   
2017-02-16 234.950 235.160 233.850 234.720    215.579   84722400     NaN   
2017-02-17 233.950 235.090 233.930 235.090    215.918   77204100     NaN   
2017-02-21 235.520 236.690 235.510 236.490    217.204   88946100     NaN   
2017-02-22 236.020 236.540 235.830 236.280    217.011   62115200     NaN   
2017-02-23 236.880 236.900 235.560 236.440    217.158   74615900     NaN   
2017-02-24 235.460 236.790 235.410 236.740    217.434   82381600     NaN   
2017-02-27 236.640 237.310 236.350 237.110    217.774   56515400     NaN   
2017-02-28 236.670 236.950 236.020 236.470    217.186   96961900     NaN   
2017-03-01 238.390 240.320 238.370 239.780    220.226  149158200     NaN   
2017-03-02 239.560 239.570 238.210 238.270    218.839   70246000     NaN   
2017-03-03 238.170 238.610 237.730 238.420    218.977   81974300     NaN   
2017-03-06 237.500 238.120 237.010 237.710    218.325   55391500     NaN   
2017-03-07 237.360 237.770 236.760 237.000    217.673   65103700     NaN   
2017-03-08 237.340 237.640 236.400 236.560    217.269   78168800     NaN   
2017-03-09 236.700 237.240 235.740 236.860    217.544   90683900     NaN   
2017-03-10 237.970 238.020 236.590 237.690    218.306   81991700     NaN   
2017-03-13 237.620 237.860 237.240 237.810    218.417   57256800     NaN   
2017-03-14 237.180 237.240 236.190 236.900    217.581   59880800     NaN   
2017-03-15 237.560 239.440 237.290 238.950    219.464   96081800     NaN   

               cci   upper  middle   lower  mom10     m1  
da